Importing Libraries

In [ ]:
import numpy as np
import os
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Conv2DTranspose
from keras.models import Model
from keras.preprocessing import image
from keras.optimizers import Adam
import tensorflow as tf
from PIL import Image

Importing Image Sets

In [ ]:
#access image directories
train_dir = 'C:/Users/GOKAY/Desktop/imagenette4/train/'
val_dir = 'C:/Users/GOKAY/Desktop/imagenette4/val/'
test_dir = 'C:/Users/GOKAY/Desktop/imagenette4/test/'

train = []
for filename in os.listdir(train_dir):
    if filename.endswith(".JPEG"):
        img = image.load_img(train_dir+filename)
        train.append(image.img_to_array(img))
train = np.array(train)

val = []
for filename in os.listdir(val_dir):
    if filename.endswith(".JPEG"):
        img = image.load_img(val_dir+filename)
        val.append(image.img_to_array(img))
val = np.array(val)

test = []
for filename in os.listdir(test_dir):
    if filename.endswith(".JPEG"):
        img = image.load_img(test_dir+filename)
        test.append(image.img_to_array(img))
test = np.array(test)

In [ ]:
print("train", train.shape)
print("val", val.shape)
print("test", test.shape)

Creating Autoencoders

In [ ]:
#the autoencoder with latent space 80x80
def first_model():
    #encoder
    input_layer = Input(shape=(160,160,3), name='INPUT')
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_layer)

    latent_layer = MaxPooling2D((2,2))(x)

    #decoder
    x = Conv2DTranspose(64, (3, 3), activation='relu', padding='same')(latent_layer)
    x = UpSampling2D((2, 2))(x)
    output_layer = Conv2D(3, (3, 3), padding='same', name="OUTPUT")(x)
    
    #creating the model
    autoencoder = Model(input_layer, output_layer)
    autoencoder.compile(optimizer=Adam(learning_rate=0.0015), loss='mse')
    
    return autoencoder

#the autoencoder with latent space 40x40
def second_model():
    #encoder
    input_layer = Input(shape=(160,160,3), name='INPUT')
    x = Conv2D(32, (3, 3), activation='leaky_relu', padding='same')(input_layer)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='leaky_relu', padding='same')(x)

    latent_layer = MaxPooling2D((2, 2))(x)

    #decoder
    x = Conv2DTranspose(64, (3, 3), activation='leaky_relu', padding='same')(latent_layer)
    x = UpSampling2D((2, 2))(x)
    x = Conv2DTranspose(32, (3, 3), activation='leaky_relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    output_layer = Conv2D(3, (3, 3), padding='same', name="OUTPUT")(x)

    #creating the model
    autoencoder = Model(input_layer, output_layer)
    autoencoder.compile(optimizer=Adam(learning_rate=0.0017), loss='mse')

    return autoencoder

#the autoencoder with 20x20 latent space
def third_model():
    #encoder
    input_layer = Input(shape=(160,160,3), name='INPUT')
    x = Conv2D(32, (3, 3), activation='leaky_relu', padding='same')(input_layer)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='leaky_relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='leaky_relu', padding='same')(x)

    latent_layer = MaxPooling2D((2, 2))(x)
    
    #decoder
    x = Conv2DTranspose(128, (3, 3), activation='leaky_relu', padding='same')(latent_layer)
    x = UpSampling2D((2, 2))(x)
    x = Conv2DTranspose(64, (3, 3), activation='leaky_relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2DTranspose(32, (3, 3), activation='leaky_relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    output_layer = Conv2D(3, (3, 3), padding='same', name="OUTPUT")(x)
    
    autoencoder = Model(input_layer, output_layer)
    autoencoder.compile(optimizer=Adam(learning_rate=0.0021), loss='mse')

    return autoencoder


Creating the callbacks

In [ ]:
def check(model_no):
    #checkpoint to save the best result
    checkpoint = tf.keras.callbacks.ModelCheckpoint(
        f'ae{model_no}.keras',          
        monitor='val_loss',       
        save_best_only=True,      
        mode='min',               
        verbose=1                 
    )

    #Setting up early stopping to stop the training when no improvment is achieved and receiving the best results(weights)
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=5,                
        verbose=1,                 
        mode='min',               
        restore_best_weights=True
    )
    return checkpoint, early_stopping


Creating Each Model and Training Them

In [ ]:
ae1 = first_model()
ae1.fit(train, train, epochs=100, batch_size=32, validation_data=(val, val), callbacks=check(1), verbose=1)

In [ ]:
ae2 = second_model()
ae2.fit(train, train, epochs=100, batch_size=32, validation_data=(val, val),callbacks=check(2), verbose=1)

In [ ]:
ae3 = third_model()
ae3.fit(train, train, epochs=100, batch_size=32, validation_data=(val, val),callbacks=check(3), verbose=1)

Compression of Images With Conventional Methods For Each Quality Chosen

In [ ]:
#Base directory for compressed images
base_output_dir = 'C:/Users/GOKAY/Desktop/compressed_images/conv/'

#Subdirectories to be created for each combination (except tiff)
formats = ['JPEG', 'WEBP', 'TIFF']
qualities = [25, 50, 75]

#Checking if already bases directory exists
if not os.path.exists(base_output_dir):
    os.makedirs(base_output_dir)

In [ ]:
#Creating subdirectories
for formt in formats:
    #Creating subdirectories for jpeg and webp
    if formt != 'TIFF':
        for quality in qualities:
            dir_path = os.path.join(base_output_dir, f'{formt} {quality}')
            if not os.path.exists(dir_path):
                os.makedirs(dir_path)
    else:
        #Subdirectory for tiff
        dir_path = os.path.join(base_output_dir, formt)
        if not os.path.exists(dir_path):
            os.makedirs(dir_path)

In [ ]:
#Processing each image file in the test set 
for filename in os.listdir(test_dir):
    if filename.lower().endswith(('.jpg')): 
        img_path = os.path.join(test_dir, filename)
        img = Image.open(img_path)

        #Comperssing and saving the image in each format and quality setting
        for formt in formats:
            base_filename = filename.rsplit('.', 1)[0]  # Strip extension
            #jpeg and webp compressions
            if formt != 'TIFF':
                for quality in qualities:
                    output_filename = f"{base_filename}.{formt.lower()}"
                    output_filepath = os.path.join(base_output_dir, f'{formt} {quality}', output_filename)
                    img.save(output_filepath, format=formt, quality=quality)
            #tiff compression
            else:
                output_filename = f"{base_filename}.{formt.lower()}"
                output_filepath = os.path.join(base_output_dir, formt, output_filename)
                img.save(output_filepath, format=formt, compression='tiff_lzw')

print("Image compression was done successfully.")
